## Initialization of the notebook with installation and import of correct versions of libraries.

In [15]:
!pip install torchinfo
!pip install brevitas
!pip install qonnx
!pip install onnxoptimizer

In [16]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [17]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May 27 19:39:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [18]:
import numpy as np
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
import seaborn as sns
import pandas as pd
import gc
import cv2

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [20]:
import brevitas.nn as qnn
from brevitas.quant import Int8ActPerTensorFloat


In [21]:
read_small = True

In [22]:
if read_small:
  !unzip drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip -d ./dataset
else:
  !unzip drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_FT_medium.zip -d ./dataset

unzip:  cannot find or open drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip, drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip.zip or drive/MyDrive/TecnosensSondaOttica/DatasetTS_MA_small.zip.ZIP.


In [23]:
datasetPath = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset"

In [24]:
%ls

drive/  sample_data/


## Setting a seed on all frameworks for reproducibility

In [25]:
# Random seed for reproducibility
seed = 1234

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Load data and preprocessing

## Generate dataset

In [12]:
labels_df = pd.read_csv(datasetPath+"/labels_df2.csv")
#if not read_small:
  #labels_df = pd.read_csv("../drive/MyDrive/TecnosensSondaOttica/labels_df_ft.csv", index_col=False)

In [13]:
X_train = []
Y_train = []
X_val = []
Y_val = []
X_test = []
Y_test = []

#dir = "DatasetTS_MA_FT_medium"
if read_small:
  dir = datasetPath+"/DatasetTS_MA_small"
directories = os.listdir(dir)
if ".DS_Store" in directories:
    directories.remove(".DS_Store")
directories = [int(dir) for dir in directories]
directories.sort()
directories

[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]

In [14]:
np.random.seed(42)
print(gc.collect())

used_angles = [-5,-4,-3,-2,-1,0,1,2,3,4,5]
if not read_small:
  used_angles = [-5, 5]

for directory in tqdm(directories):
  if directory in used_angles:
    files = os.listdir(dir + '/' + str(directory))
    files.sort()
    files = list(reversed(files))[:1100]
    for file in tqdm(files):
      try:
        distance_str = file.split('-')[0]
        distance_val = int(distance_str)
      except:
        print(f"Impossibile estrarre distanza da {file}")
        continue

      match = labels_df[(labels_df['distance'] == distance_val) & (labels_df['angle'] == directory)]
      if len(match) == 0:
        print(f"Nessuna label trovata per file {file} con distanza {distance_val} e angolo {directory}")
        continue

      train_val_test = np.random.randint(0,100)  # 80-5-15 split

      image = cv2.imread(dir + "/" + str(directory) + "/" + file)
      image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      if train_val_test < 80:
        image = cv2.resize(image, (300,300))
        X_train.append(image.astype('float16')/255)
        Y_train.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
        if len(X_train) != len(Y_train):
          print("male", file)
          print(len(X_train))
          print(len(Y_train))
      elif train_val_test < 85:
        image = cv2.resize(image, (300,300))
        X_val.append(image.astype('float16')/255)
        Y_val.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
      else:
        image = cv2.resize(image, (300,300))
        X_test.append(image.astype('float16')/255)
        Y_test.append([match.iloc[0]['distance'], match.iloc[0]['angle']])
  else:
    print("Angle " + str(directory) + " skipped")


4287


  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

...

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

  0%|          | 0/1100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
gc.collect()

X_train = np.array(X_train)
Y_train = np.array(Y_train, dtype="float32")
print("Shape of Train set's images: " + str(X_train.shape))
print("Shape of Train set's labels: " + str(Y_train.shape))

X_val = np.array(X_val)
Y_val = np.array(Y_val, dtype="float32")
print("Shape of Val set's images: " + str(X_val.shape))
print("Shape of Val set's labels: " + str(Y_val.shape))

X_test = np.array(X_test)
Y_test = np.array(Y_test, dtype="float32")
print("Shape of Test set's images: " + str(X_test.shape))
print("Shape of Test set's labels: " + str(Y_test.shape))

## Labels Normalization

In [ ]:
mean = np.mean(Y_train[:,0])
std = np.std(Y_train[:,0])
Y_train[:,0] = (Y_train[:,0] - mean)/std
Y_val[:,0] = (Y_val[:,0] - mean)/std
Y_test[:,0] = (Y_test[:,0] - mean)/std

In [ ]:
np.max(Y_train[:,0]), np.min(Y_train[:,0]), np.mean(Y_train[:,0])

In [ ]:
mean, std

## Save dataset

In [ ]:
save_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

os.makedirs(save_path, exist_ok=True)

np.save(os.path.join(save_path, "X_train.npy"), X_train)
np.save(os.path.join(save_path, "Y_train.npy"), Y_train)
np.save(os.path.join(save_path, "X_val.npy"), X_val)
np.save(os.path.join(save_path, "Y_val.npy"), Y_val)
np.save(os.path.join(save_path, "X_test.npy"), X_test)
np.save(os.path.join(save_path, "Y_test.npy"), Y_test)


## Open Dataset

In [26]:
load_path = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Dataset/TensorflowDataset"

X_train = np.load(os.path.join(load_path, "X_train.npy"))
Y_train = np.load(os.path.join(load_path, "Y_train.npy"))
X_val = np.load(os.path.join(load_path, "X_val.npy"))
Y_val = np.load(os.path.join(load_path, "Y_val.npy"))
X_test = np.load(os.path.join(load_path, "X_test.npy"))
Y_test = np.load(os.path.join(load_path, "Y_test.npy"))


In [27]:
def resize_set(X):
    X = X.astype(np.float32)
    resized = np.zeros((X.shape[0], 128, 128), dtype=np.float32)
    for i in range(X.shape[0]):
        resized[i] = cv2.resize(X[i], (128, 128), interpolation=cv2.INTER_AREA)
    return resized

X_train = resize_set(X_train)
X_val = resize_set(X_val)
X_test = resize_set(X_test)

print("Resized shapes:")
print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("X_test:", X_test.shape)


Resized shapes:
X_train: (9673, 128, 128)
X_val: (590, 128, 128)
X_test: (1837, 128, 128)


In [28]:
class OpticalDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # aggiunge il canale
        self.Y = torch.tensor(Y[:,0], dtype=torch.float32)  # solo la distanza normalizzata

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

train_dataset = OpticalDataset(X_train, Y_train)
val_dataset = OpticalDataset(X_val, Y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [29]:
test_dataset = OpticalDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Definition of the models



```
        self.conv1 = qnn.QuantConv2d(1, 8*x, kernel_size=3, stride=1, padding=1,
                                     weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                     input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
        self.conv2 = qnn.QuantConv2d(8*x, 12*x, kernel_size=3, stride=1, padding=1,
                                     weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                     input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
        self.conv3 = qnn.QuantConv2d(12*x, 16*x, kernel_size=3, stride=1, padding=1,
                                     weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                     input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
        self.conv4 = qnn.QuantConv2d(16*x, 20*x, kernel_size=3, stride=1, padding=1,
                                     weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                     input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
        self.conv5 = qnn.QuantConv2d(20*x, 24*x, kernel_size=3, stride=1, padding=1,
                                     weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                     input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
```



In [30]:
import brevitas.nn as qnn
from brevitas.quant import Int8ActPerTensorFloat
import torch

x = 2
weight_bit_width = 8
InOutQuant = False

class StandardModel(torch.nn.Module):
    def __init__(self, input_shape=(1, 128, 128)):
        super(StandardModel, self).__init__()
        torch.manual_seed(42)


        act_bit_width = weight_bit_width

        if InOutQuant==False:
            self.conv1 = qnn.QuantConv2d(1, 8*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width)
            self.conv2 = qnn.QuantConv2d(8*x, 12*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width)
            self.conv3 = qnn.QuantConv2d(12*x, 16*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width)
            self.conv4 = qnn.QuantConv2d(16*x, 20*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width)
            self.conv5 = qnn.QuantConv2d(20*x, 24*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width)
        else:
            self.conv1 = qnn.QuantConv2d(1, 8*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                        input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
            self.conv2 = qnn.QuantConv2d(8*x, 12*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                        input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
            self.conv3 = qnn.QuantConv2d(12*x, 16*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                        input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
            self.conv4 = qnn.QuantConv2d(16*x, 20*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                        input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)
            self.conv5 = qnn.QuantConv2d(20*x, 24*x, kernel_size=3, stride=1, padding=1,
                                        weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width,
                                        input_quant=Int8ActPerTensorFloat, output_quant=Int8ActPerTensorFloat)

        # Activations
        self.relu1 = qnn.QuantReLU(bit_width=weight_bit_width)
        self.relu2 = qnn.QuantReLU(bit_width=weight_bit_width)
        self.relu3 = qnn.QuantReLU(bit_width=weight_bit_width)
        self.relu4 = qnn.QuantReLU(bit_width=weight_bit_width)
        self.relu5 = qnn.QuantReLU(bit_width=weight_bit_width)
        self.relu_fc1 = qnn.QuantReLU(bit_width=weight_bit_width)

        # Pooling
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.global_pool = torch.nn.AdaptiveAvgPool2d(1)

        # Fully connected layers
        self.fc1 = qnn.QuantLinear(24*x, 16*x, weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width)
        self.dropout = torch.nn.Dropout(0.3)
        self.out = qnn.QuantLinear(16*x, 1, weight_bit_width=weight_bit_width, bias_bit_width=weight_bit_width)

    def forward(self, x):
        x = self.pool(self.relu1(self.conv1(x)))  # 128 → 64
        x = self.pool(self.relu2(self.conv2(x)))  # 64 → 32
        x = self.pool(self.relu3(self.conv3(x)))  # 32 → 16
        x = self.pool(self.relu4(self.conv4(x)))  # 16 → 8
        x = self.pool(self.relu5(self.conv5(x)))  # 8 → 4
        x = self.global_pool(x)                   # 4x4 → 1x1
        x = torch.flatten(x, 1)
        x = self.relu_fc1(self.fc1(x))
        x = self.dropout(x)
        x = self.out(x)
        return x


# Training

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = StandardModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.L1Loss()  # Mean Absolute Error

best_val_loss = float('inf')
patience, patience_counter = 10, 0

for epoch in range(1, 1001):
    model.train()
    train_loss = 0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
        #print(f"Input shape: {inputs.shape}")
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()  # puoi anche accumulare per media

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device).unsqueeze(1)
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
    val_loss /= len(val_loader)

    print(f"Epoch {epoch:03d} - Train Loss: {train_loss:.6f} - Val Loss: {val_loss:.6f}", end='')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience_counter = 0
        print("  --> New best model saved!")
    else:
        patience_counter += 1
        print(f"  --> No improvement ({patience_counter}/{patience})")

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


/usr/local/lib/python3.11/dist-packages/torch/_tensor.py:1624: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /pytorch/c10/core/TensorImpl.h:1935.)
  return super().rename(names)


Epoch 001 - Train Loss: 0.656007 - Val Loss: 0.777793  --> New best model saved!
Epoch 002 - Train Loss: 0.288238 - Val Loss: 0.333341  --> New best model saved!
Epoch 003 - Train Loss: 0.266925 - Val Loss: 0.308814  --> New best model saved!
Epoch 004 - Train Loss: 0.166705 - Val Loss: 0.196518  --> New best model saved!
Epoch 005 - Train Loss: 0.194782 - Val Loss: 0.187546  --> New best model saved!
Epoch 006 - Train Loss: 0.278415 - Val Loss: 0.147182  --> New best model saved!
Epoch 007 - Train Loss: 0.191941 - Val Loss: 0.158453  --> No improvement (1/10)
Epoch 008 - Train Loss: 0.135919 - Val Loss: 0.115599  --> New best model saved!
Epoch 009 - Train Loss: 0.232055 - Val Loss: 0.103828  --> New best model saved!
Epoch 010 - Train Loss: 0.123219 - Val Loss: 0.093166  --> New best model saved!
Epoch 011 - Train Loss: 0.192288 - Val Loss: 0.112853  --> No improvement (1/10)
Epoch 012 - Train Loss: 0.118179 - Val Loss: 0.069922  --> New best model saved!
Epoch 013 - Train Loss: 0.27

## Save the model and collect garbage for resource issues of Colab's RAM

In [32]:
%ls

best_model.pth  drive/  sample_data/


In [33]:
from brevitas.export import export_qonnx

class QONNXExporter:
    def __init__(self, model, model_name, input_shape, export_path):
        self.model = model
        self.model_name = model_name
        self.input_shape = input_shape
        self.export_path = export_path
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def export(self):
        self.model.to(self.device)
        self.model.eval()
        dummy_input = torch.randn(self.input_shape).to(self.device)
        export_qonnx(self.model, dummy_input, self.export_path)
        print(f"QONNX model exported to: {self.export_path}\n")


In [34]:
import os

EXPORT_DIR = "drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Weights/Size128"
if InOutQuant==True:
    MODEL_NAME = f"standardRegressionModel_brevitas_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth{weight_bit_width}_InOutQuant_X{x}"
else:
    MODEL_NAME = f"standardRegressionModel_brevitas_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth{weight_bit_width}_NOInOutQuant_X{x}"


SAVE_DIR = os.path.join(EXPORT_DIR, MODEL_NAME)
os.makedirs(SAVE_DIR, exist_ok=True)

In [35]:


#model.save(os.path.join(SAVE_DIR, f"{MODEL_NAME}.keras"))
torch.save(model.state_dict(), os.path.join(SAVE_DIR, f"{MODEL_NAME}.pth"))
qonnx_export_path = os.path.join(SAVE_DIR, f"{MODEL_NAME}.qonnx")
exporter = QONNXExporter(model, MODEL_NAME, input_shape=(1, 1, 128, 128), export_path=qonnx_export_path)
exporter.export()
gc.collect()

QONNX model exported to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Weights/Size128/standardRegressionModel_brevitas_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth8_NOInOutQuant_X2/standardRegressionModel_brevitas_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth8_NOInOutQuant_X2.qonnx



73

## Predictions

In [36]:
from torchinfo import summary  # Assicurati che torchinfo sia installato

# Riassunto del modello
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = StandardModel().to(device)
summary(model, input_size=(1, 1, 128, 128))

# Dettagli layer
for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear, nn.MaxPool2d, nn.Dropout)):
        print(f"Layer: {name}")
        if isinstance(layer, nn.Conv2d):
            print(f"  Conv2d -> Filters: {layer.out_channels}, Kernel: {layer.kernel_size}, Stride: {layer.stride}, Padding: {layer.padding}")
        if isinstance(layer, nn.Linear):
            print(f"  Linear -> Units: {layer.out_features}")
        if isinstance(layer, nn.MaxPool2d):
            print(f"  MaxPool2d -> Pool size: {layer.kernel_size}")
        if isinstance(layer, nn.Dropout):
            print(f"  Dropout -> Rate: {layer.p}")
        print()

# Predizioni su test set
model.load_state_dict(torch.load(os.path.join(SAVE_DIR, f"{MODEL_NAME}.pth")))
#model.load_state_dict(torch.load('best_model.pth'))
model.to(device)
model.eval()

with torch.no_grad():
    preds = []
    targets = []
    for inputs, target in test_loader:
        inputs = inputs.to(device)
        output = model(inputs).cpu().numpy()
        preds.extend(output)
        targets.extend(target.numpy())

preds = np.array(preds).squeeze()
targets = np.array(targets)

# Rescaling
mean = np.float32(5901007.5)
std = np.float32(633870.7)
y_test_pred_rescaled = preds * std + mean
Y_test_rescaled = targets * std + mean

# Errori
errors = np.abs(Y_test_rescaled - y_test_pred_rescaled) / 1000  # micrometri
avg_error = np.mean(errors)
max_error = np.max(errors)
min_error = np.min(errors)

# Salvataggio su file
output_text = (
    f"Average error: {avg_error} micrometers\n"
    f"Maximum error: {max_error} micrometers\n"
    f"Minimum error: {min_error} micrometers\n"
)
output_file = os.path.join(SAVE_DIR, "error_report.txt")
with open(output_file, "w") as f:
    f.write(output_text)

# Stampa
print(f"Average error is {avg_error:.6f} micrometers")
print(f"Maximum error is {max_error:.3f} micrometers")
print(f"Minimum error is {min_error:.4f} micrometers")


Layer: conv1
  Conv2d -> Filters: 16, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv2
  Conv2d -> Filters: 24, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv3
  Conv2d -> Filters: 32, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv4
  Conv2d -> Filters: 40, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv5
  Conv2d -> Filters: 48, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: pool
  MaxPool2d -> Pool size: 2

Layer: fc1
  Linear -> Units: 32

Layer: dropout
  Dropout -> Rate: 0.3

Layer: out
  Linear -> Units: 1

Average error is 30.348892 micrometers
Maximum error is 148.993 micrometers
Minimum error is 0.0105 micrometers


In [37]:
print("\nFirst 200 predictions vs ground truth:\n")
print("Idx\tPredicted [µm]\tGT [µm]\t\tAbsError [µm]")
for i in range(min(200, len(y_test_pred_rescaled))):
    pred = y_test_pred_rescaled[i]
    gt = Y_test_rescaled[i]
    err = abs(pred - gt)
    print(f"{i:03d}\t{pred:.2f}\t\t{gt:.2f}\t\t{err:.2f}")



First 200 predictions vs ground truth:

Idx	Predicted [µm]	GT [µm]		AbsError [µm]
000	6849006.50		6998000.00		148993.50
001	6845809.50		6986000.00		140190.50
002	6844258.00		6980000.00		135742.00
003	6841834.50		6978000.00		136165.50
004	6836988.00		6966000.00		129012.00
005	6825157.00		6944000.00		118843.00
006	6824732.00		6940000.00		115268.00
007	6821495.00		6934000.00		112505.00
008	6802881.50		6896000.00		93118.50
009	6794865.00		6888000.00		93135.00
010	6792938.00		6880000.00		87062.00
011	6769388.00		6846000.00		76612.00
012	6762659.00		6840000.00		77341.00
013	6758568.00		6830000.00		71432.00
014	6757463.50		6826000.00		68536.50
015	6743446.50		6816000.00		72553.50
016	6732729.00		6798000.00		65271.00
017	6717236.50		6784000.00		66763.50
018	6711616.00		6778000.00		66384.00
019	6707310.50		6772000.00		64689.50
020	6671116.00		6732000.00		60884.00
021	6667982.00		6728000.00		60018.00
022	6664293.00		6724000.00		59707.00
023	6666725.50		6722000.00		55274.50
024	6659969.50		67160

## Add Double Neurons

In [38]:
# Aggiunta secondo neurone mantenendo il primo invariato
import copy
import torch.nn as nn
import brevitas.nn as qnn
from brevitas.quant import Int8ActPerTensorFloat

# Salva stato del vecchio modello
model_old = model  # il modello già caricato con output = 1

# Crea nuovo modello con stesso backbone ma output a 2 neuroni
model_new = copy.deepcopy(model_old)
model_new.out = qnn.QuantLinear(16*x, 2, weight_bit_width=8, bias_bit_width=8)

# Copia i pesi del primo neurone
with torch.no_grad():
    model_new.out.weight[0] = model_old.out.weight[0]
    model_new.out.bias[0] = model_old.out.bias[0]
    model_new.out.weight[1].zero_()
    model_new.out.bias[1].zero_()

# Invia su device
model = model_new.to(device)
model.eval()

# Riassunto
from torchinfo import summary
summary(model, input_size=(1, 1, 128, 128))

for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.Linear, nn.MaxPool2d, nn.Dropout)):
        print(f"Layer: {name}")
        if isinstance(layer, nn.Conv2d):
            print(f"  Conv2d -> Filters: {layer.out_channels}, Kernel: {layer.kernel_size}, Stride: {layer.stride}, Padding: {layer.padding}")
        if isinstance(layer, nn.Linear):
            print(f"  Linear -> Units: {layer.out_features}")
        if isinstance(layer, nn.MaxPool2d):
            print(f"  MaxPool2d -> Pool size: {layer.kernel_size}")
        if isinstance(layer, nn.Dropout):
            print(f"  Dropout -> Rate: {layer.p}")
        print()

# Predizioni su test set usando solo il primo neurone
with torch.no_grad():
    preds = []
    targets = []
    for inputs, target in test_loader:
        inputs = inputs.to(device)
        output = model(inputs).cpu().numpy()
        preds.extend(output[:, 0])  # usa solo il primo neurone
        targets.extend(target.numpy())

preds = np.array(preds).squeeze()
targets = np.array(targets)

# Rescaling
mean = np.float32(5901007.5)
std = np.float32(633870.7)
y_test_pred_rescaled = preds * std + mean
Y_test_rescaled = targets * std + mean

# Errori
errors = np.abs(Y_test_rescaled - y_test_pred_rescaled) / 1000  # micrometri
avg_error = np.mean(errors)
max_error = np.max(errors)
min_error = np.min(errors)

# Salvataggio su file
output_text = (
    f"Average error: {avg_error} micrometers\n"
    f"Maximum error: {max_error} micrometers\n"
    f"Minimum error: {min_error} micrometers\n"
)

# Percorsi di salvataggio
if InOutQuant==True:
    MODEL_NAME = f"standardRegressionModel_brevitas_doubleNeuronsAddedPostTrain_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth{weight_bit_width}_InOutQuant_X{x}"
else:
    MODEL_NAME = f"standardRegressionModel_brevitas_doubleNeuronsAddedPostTrain_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth{weight_bit_width}_NOInOutQuant_X{x}"


SAVE_DIR = os.path.join(EXPORT_DIR, MODEL_NAME)
os.makedirs(SAVE_DIR, exist_ok=True)

output_file = os.path.join(SAVE_DIR, "error_report.txt")
with open(output_file, "w") as f:
    f.write(output_text)

print(f"Average error is {avg_error:.6f} micrometers")
print(f"Maximum error is {max_error:.3f} micrometers")
print(f"Minimum error is {min_error:.4f} micrometers")


Layer: conv1
  Conv2d -> Filters: 16, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv2
  Conv2d -> Filters: 24, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv3
  Conv2d -> Filters: 32, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv4
  Conv2d -> Filters: 40, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: conv5
  Conv2d -> Filters: 48, Kernel: (3, 3), Stride: (1, 1), Padding: (1, 1)

Layer: pool
  MaxPool2d -> Pool size: 2

Layer: fc1
  Linear -> Units: 32

Layer: dropout
  Dropout -> Rate: 0.3

Layer: out
  Linear -> Units: 2

Average error is 30.348896 micrometers
Maximum error is 148.993 micrometers
Minimum error is 0.0105 micrometers


In [39]:
from brevitas.export import export_qonnx
import os
import gc

class QONNXExporter:
    def __init__(self, model, model_name, input_shape, export_path):
        self.model = model
        self.model_name = model_name
        self.input_shape = input_shape
        self.export_path = export_path
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def export(self):
        self.model.to(self.device)
        self.model.eval()
        dummy_input = torch.randn(self.input_shape).to(self.device)
        export_qonnx(self.model, dummy_input, self.export_path)
        print(f"QONNX model exported to: {self.export_path}\n")


# Salva il modello modificato
torch.save(model.state_dict(), os.path.join(SAVE_DIR, f"{MODEL_NAME}.pth"))

# Esporta in QONNX
qonnx_export_path = os.path.join(SAVE_DIR, f"{MODEL_NAME}.qonnx")
exporter = QONNXExporter(model, MODEL_NAME, input_shape=(1, 1, 128, 128), export_path=qonnx_export_path)
exporter.export()
gc.collect()


QONNX model exported to: drive/MyDrive/HPPS_Nico/HPPS/Project/ModelRegression/Weights/Size128/standardRegressionModel_brevitas_doubleNeuronsAddedPostTrain_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth8_NOInOutQuant_X2/standardRegressionModel_brevitas_doubleNeuronsAddedPostTrain_datasetTensorflow01_nameMappingInOrder_1100Class_80train5val15test_lr1e-4_bitWidth8_NOInOutQuant_X2.qonnx



13706